# 1. Introduction to the Data

In the last mission, we used a data set on U.S. income from the 1994 census; we'll continue using it here. It contains information on marital status, age, type of work, and more. The target column, high_income, indicates a salary less than or equal to 50k per year (0), or more than 50k per year (1).

You can download the data from the [University of California, Irvine's website](http://archive.ics.uci.edu/ml/datasets/Adult).

# 2. Overview of the ID3 Algorithm

In the last mission, we learned about the basics of decision trees, including entropy and information gain. In this mission, we'll build on those concepts to construct a full decision tree in Python and use it to make predictions.

We'll use the [ID3 Algorithm](https://en.wikipedia.org/wiki/ID3_algorithm) for constructing decision trees to accomplish this. This algorithm involves `recursion` and an understanding of `time complexity`. If you're unfamiliar with these topics, we suggest trying our Data Structures and Algorithms course. We also suggest learning about lambda functions through our lambda functions mission.

In general, recursion is the process of splitting a large problem into smaller chunks. Recursive functions will call themselves, then combine the results into a final output.

Building a tree is a perfect use case for recursive algorithms. At each node, we'll call a recursive function that will split the data into two branches. Each branch will lead to a node, and the function will call itself to build the tree out.

Exploring the pseudocode for an algorithm is a good way to understand it better before trying to code it.

`def id3(data, target, columns)
    1 Create a node for the tree
    2 If all values of the target attribute are 1, Return the node, with label = 1
    3 If all values of the target attribute are 0, Return the node, with label = 0
    4 Using information gain, find A, the column that splits the data best
    5 Find the median value in column A
    6 Split column A into values below or equal to the median (0), and values above the median (1)
    7 For each possible value (0 or 1), vi, of A,
    8    Add a new tree branch below Root that corresponds to rows of data where A = vi
    9    Let Examples(vi) be the subset of examples that have the value vi for A
   10    Below this new branch add the subtree id3(data[A==vi], target, columns)
   11 Return Root`

# 3. Walking Through an Example of the ID3 Algorithm

# 4. Determining the Column to Split On

In [1]:
import pandas as pd
income=pd.read_csv('income.csv')

columns = ["age", "workclass", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]

In [2]:
from Introduction_to_Decision_Trees import calc_entropy,calc_information_gain


def find_best_column(data, target_name, columns):
    information_gains = []
    # Loop through and compute information gains
    for col in columns:
        information_gain = calc_information_gain(data, col, "high_income")
        information_gains.append(information_gain)

    # Find the name of the column with the highest gain
    highest_gain_index = information_gains.index(max(information_gains))
    highest_gain = columns[highest_gain_index]
    return highest_gain

income_split = find_best_column(income, "high_income", columns)

   age          workclass  fnlwgt   education  education_num  \
0   39          State-gov   77516   Bachelors             13   
1   50   Self-emp-not-inc   83311   Bachelors             13   
2   38            Private  215646     HS-grad              9   
3   53            Private  234721        11th              7   
4   28            Private  338409   Bachelors             13   

        marital_status          occupation    relationship    race      sex  \
0        Never-married        Adm-clerical   Not-in-family   White     Male   
1   Married-civ-spouse     Exec-managerial         Husband   White     Male   
2             Divorced   Handlers-cleaners   Not-in-family   White     Male   
3   Married-civ-spouse   Handlers-cleaners         Husband   Black     Male   
4   Married-civ-spouse      Prof-specialty            Wife   Black   Female   

   capital_gain  capital_loss  hours_per_week  native_country high_income  
0          2174             0              40   United-States   

ValueError: invalid literal for int() with base 10: ' <=50K'

# 5. Creating a Simple Recursive Algorithm

Let's build up to making the full id3() function by creating a simpler algorithm that we can extend. Here's what that algorithm looks like in pseudocode:

`def id3(data, target, columns)
    1 Create a node for the tree
    2 If all values of the target attribute are 1, add 1 to counter_1
    3 If all values of the target attribute are 0, add 1 to counter_0
    4 Using information gain, find A, the column that splits the data best
    5 Find the median value in column A
    6 Split A into values below or equal to the median (0), and values above the median (1)
    7 For each possible value (0 or 1), vi, of A,
    8    Add a new tree branch below Root that corresponds to rows of data where A = vi
    9    Let Examples(vi) be the subset of examples that have the value vi for A
   10    Below this new branch, add the subtree id3(data[A==vi], target, columns)
   11 Return Root`

In [6]:
# We'll use lists to store our labels for nodes (when we find them)
# Lists can be accessed inside our recursive function, whereas integers can't.  
# Look at the python missions on scoping for more information on this topic
label_1s = []
label_0s = []

import pandas
    
# Create the data set that we used in the example on the last screen
data = pandas.DataFrame([
    [0,20,0],
    [0,60,2],
    [0,40,1],
    [1,25,1],
    [1,35,2],
    [1,55,1]
    ])
# Assign column names to the data
data.columns = ["high_income", "age", "marital_status"]

# Call the function on our data to set the counters properly
id3(data, "high_income", ["age", "marital_status"])
label_1s = []
label_0s = []

def id3(data, target, columns):
    unique_targets = pandas.unique(data[target])

    if len(unique_targets) == 1:
        if 0 in unique_targets:
            label_0s.append(0)
        elif 1 in unique_targets:
            label_1s.append(1)
        return
    
    best_column = find_best_column(data, target, columns)
    column_median = data[best_column].median()
    
    left_split = data[data[best_column] <= column_median]
    right_split = data[data[best_column] > column_median]
    
    for split in [left_split, right_split]:
        id3(split, target, columns)


id3(data, "high_income", ["age", "marital_status"])

# 6. Storing the Tree

Now we can store the entire tree, rather than the leaf labels only. We'll use nested dictionaries to do this. We can represent the root node with a dictionary, and branches with the keys left and right. We'll store the column we're splitting on as the key column, and the median value as the key median. Finally, we can store the label for a leaf as the key label. We'll also number each node as we go along using the number key.

Here's what the dictionary for the decision tree will look like:

`{  
   "left":{  
      "left":{  
         "left":{  
            "number":4,
            "label":0
         },
         "column":"age",
         "median":22.5,
         "number":3,
         "right":{  
            "number":5,
            "label":1
         }
      },
      "column":"age",
      "median":25.0,`

In order to keep track of the tree, we'll need to make some modifications to id3(). The first modification involves changing the definition to pass in the tree dictionary:

`def id3(data, target, columns, tree)
    1 Create a node for the tree
    2 Number the node
    3 If all of the values of the target attribute are 1, assign 1 to the label key in tree
    4 If all of the values of the target attribute are 0, assign 0 to the label key in tree
    5 Using information gain, find A, the column that splits the data best
    6 Find the median value in column A
    7 Assign the column and median keys in tree
    8 Split A into values less than or equal to the median (0), and values above the median (1)
    9 For each possible value (0 or 1), vi, of A,
   10    Add a new tree branch below Root that corresponds to rows of data where A = vi
   11    Let Examples(vi) be the subset of examples that have the value vi for A
   12    Create a new key with the name corresponding to the side of the split (0=left, 1=right).  The value of this key should be an empty dictionary.
   13    Below this new branch, add the subtree id3(data[A==vi], target, columns, tree[split_side])
   14 Return Root`

This process will continue building up the nested dictionary. We'll be able to access the entire dictionary using the variable tree we define before the function. Think of each recursive call as building a piece of the tree, which we can then access after all of the functions have terminated.

In [7]:
# Create a dictionary to hold the tree  
# It has to be outside of the function so we can access it later
tree = {}

# This list will let us number the nodes  
# It has to be a list so we can access it inside the function
nodes = []

def id3(data, target, columns, tree):
    unique_targets = pandas.unique(data[target])
    nodes.append(len(nodes) + 1)
    tree["number"] = nodes[-1]

    if len(unique_targets) == 1:
        if 0 in unique_targets:
            tree["label"] = 0
        elif 1 in unique_targets:
            tree["label"] = 1
        return
    
    best_column = find_best_column(data, target, columns)
    column_median = data[best_column].median()
    
    tree["column"] = best_column
    tree["median"] = column_median
    
    left_split = data[data[best_column] <= column_median]
    right_split = data[data[best_column] > column_median]
    split_dict = [["left", left_split], ["right", right_split]]
    
    for name, split in split_dict:
        tree[name] = {}
        id3(split, target, columns, tree[name])


id3(data, "high_income", ["age", "marital_status"], tree)

# 8. Printing Labels for a More Attractive Tree

The tree dictionary shows all of the relevant information, but it doesn't look very nice. We can fix its appearance by printing it out in a nicer format.

To do this, we'll need to recursively iterate through our tree dictionary. Any dictionary that has a label key is a leaf. Whenever we find one, we'll print out the label. Otherwise, we'll loop through the tree's left and right keys and recursively call the same function.

We also need to keep track of a depth variable. This variable will allow us to use indentation to indicate the order of the nodes. Before we print anything out, we'll prefix it with the number of spaces corresponding to the depth variable.

Here's the pseudocode:

`def print_node(tree, depth):
    1 Check for the presence of the "label" key in the tree
    2     If found, print the label and return
    3 Print out the tree's "column" and "median" keys
    4 Iterate through the tree's "left" and "right" keys
    5     Recursively call print_node(tree[key], depth+1)`

In [8]:
def print_with_depth(string, depth):
    # Add space before a string
    prefix = "    " * depth
    # Print a string, and indent it appropriately
    print("{0}{1}".format(prefix, string))
    
    
def print_node(tree, depth):
    # Check for the presence of "label" in the tree
    if "label" in tree:
        # If found, then this is a leaf, so print it and return
        print_with_depth("Leaf: Label {0}".format(tree["label"]), depth)
        # This is critical -- without it, you'll get infinite recursion
        return
    # Print information about what the node is splitting on
    print_with_depth("{0} > {1}".format(tree["column"], tree["median"]), depth)
    
    # Create a list of tree branches
    branches = [tree["left"], tree["right"]]
        
    # Insert code here to recursively call print_node on each branch
    # Don't forget to increment depth when you pass it in

print_node(tree, 0)
def print_node(tree, depth):
    if "label" in tree:
        print_with_depth("Leaf: Label {0}".format(tree["label"]), depth)
        return
    print_with_depth("{0} > {1}".format(tree["column"], tree["median"]), depth)
    for branch in [tree["left"], tree["right"]]:
        print_node(branch, depth+1)

print_node(tree, 0)

age > 37.5
age > 37.5
    age > 25.0
        age > 22.5
            Leaf: Label 0
            Leaf: Label 1
        Leaf: Label 1
    age > 55.0
        age > 47.5
            Leaf: Label 0
            Leaf: Label 1
        Leaf: Label 0


# 10. Making Predictions Automatically

Let's write a function that makes predictions automatically. All we need to do is follow the split points we've already defined with a new row.

Here's the pseudocode:

`def predict(tree, row):
    1 Check for the presence of "label" in the tree dictionary
    2    If found, return tree["label"]
    3 Extract tree["column"] and tree["median"]
    4 Check whether row[tree["column"]] is less than or equal to tree["median"]
    5    If it's less than or equal, call predict(tree["left"], row) and return the result
    6    If it's greater, call predict(tree["right"], row) and return the result`
    
The major difference here is that we're returning values. Because we're only calling the function recursively once in each iteration (we only go "down" a single branch), we can return a single value up the chain of recursion. This will let us get a value back when we call the function.

In [9]:
def predict(tree, row):
    if "label" in tree:
        return tree["label"]
    
    column = tree["column"]
    median = tree["median"]
    
    # Insert code here to check whether row[column] is less than or equal to median
    # If it's less than or equal, return the result of predicting on the left branch of the tree
    # If it's greater, return the result of predicting on the right branch of the tree
    # Remember to use the return statement to return the result!

# Print the prediction for the first row in our data
print(predict(tree, data.iloc[0]))
def predict(tree, row):
    if "label" in tree:
        return tree["label"]
    
    column = tree["column"]
    median = tree["median"]
    if row[column] <= median:
        return predict(tree["left"], row)
    else:
        return predict(tree["right"], row)

print(predict(tree, data.iloc[0]))

None
0


# 11. Making Multiple Predictions

In [10]:
new_data = pandas.DataFrame([
    [40,0],
    [20,2],
    [80,1],
    [15,1],
    [27,2],
    [38,1]
    ])
# Assign column names to the data
new_data.columns = ["age", "marital_status"]

def batch_predict(tree, df):
    # Insert your code here
    pass

predictions = batch_predict(tree, new_data)
def batch_predict(tree, df):
    return df.apply(lambda x: predict(tree, x), axis=1)

predictions = batch_predict(tree, new_data)